# TRABALHO AVALIATIVO 2 - INTELIGÊNCIA ARTIFICIAL
**Acadêmicos:**
- Guilherme Farias Stefani
- Henrique Baptista de Oliveira
- Henrique Cardoso Zanette
- Jhonata Saraiva Peres
- Lucas Dellatorre de Freitas


___

**Bibliotecas python nescessarias para utilização deste notebook:**

- **Pandas:** Para leitura do DataSet

- **Unidecode:** Para normalização dos textos

- **Regex:** Para normalização dos textos
     
- **Nltk:** Para executar os algortmos de machine learning

**Rodar no powershell como administrador:**

> - pip install pandas
> - pip install unidecode
> - pip install regex
> - pip install nltk

In [14]:
import pandas as pd
import regex as re
import unidecode
import nltk

- Localização dos arquivos de resources

In [13]:
filenameDataSet = 'resources/DataSet.xlsx'
filenameRegexToBeApplied =  'resources/regexToBeApplied'
filenameStopWords = 'resources/stopWords'
fileNameDeParas = 'resources/dePara'

- iNICIALIZAÇÃO DO DATASET

In [20]:
dataSet = pd.ExcelFile(filenameDataSet)

In [24]:
dataSetRaw = pd.read_excel(dataSet, sheet_name='DebateGovRS')

- RTs (texto do rt)

In [25]:
retweets = pd.read_excel(dataSet, sheet_name='RT')

In [27]:
rotulados = pd.read_excel(dataSet, sheet_name='ROTULADO')

**Classificação e Limpeza dos Dados**

- Inicialmente, retiramos todos os retweets presentes no dataset;

- Em seguida, definimos os tipos de classificação dos dados, conforme abaixo:
>   - **Contexto:** Precisa de contexto para ser entendido
>   - **Positivo:**  Mensagens positivas: Elogios, enaltecimento de candidato, etc.
>   - **Negativo:** Mensagens negativas: Ofensas, xingamentos, desmerecer propostas.
>   - **Ironia:** sentido contrario da mensagem
>   - **Neutro:** nem negativo nem positivo. Somente descritivo (narração de fatos)

- Após a definição dos tipos de classificação, começamos a classificar individualmente e manualmente, um por um;

- Depois da classificação individual, realizamos uma revisão geral em grupo, onde foram ponderadas e debatidas as classificações, também de forma manual, percorrendo por todas as linhas do dataset;

- Ao final da classificação, inicializamos a normalização dos dados, substituindo as abreviações pelas palavras de referência;

**Criação dos conjuntos de treino, teste e validação**

- A partir de todo o conjunto de dados classificado, foram mantidos apenas os valores positivos e negativos (Ironias e confusões foram removidas)

- Ordenamos por ordem alfabetica para retirar os textos duplicados do conjunto de dados

- Utilizando o tipo de dado com menor amostras (POSITIVO), foi determinado 70% de suas linhas, a quantidade necessária para o conjunto de treino.

- Antes de obter os dados e alocar-los no conjunto de treino, sua ordenação foi aleatorizada para abrangir uma quantidade maior de textos.

- Após aleatorizar foram colocados no conjunto de treino, com quantidades iguais de positivos e negativos.

- Posteriormente foram criados os conjuntos de testes e validação, ambos com 15% cada, do total de amostras do tipo POSITIVO (com menor amostras), com os valores também ordenados de forma aleatória, mas balanceados entre positivos e negativos

In [12]:
def toLower(input):
    return input.lower()

def removeLinesBreaks(input):
    return re.sub('\\n', ' ', input, re.IGNORECASE, re.MULTILINE)

def removeDoubleSpaces(input):
    return re.sub('\\s\\s', '', input, re.IGNORECASE, re.MULTILINE)

def aplyRegex(input):
    regexs = open(filenameRegexToBeApplied, 'r', encoding='UTF-8', newline='\n')
    for regex in regexs.readlines():  
        teste = re.sub(regex, ' ', input, re.IGNORECASE, re.MULTILINE)
        input = teste
    return input





- Faz o de para (explicar brevemente o de para)

In [ ]:
def dePara(input):
    deParas = open(fileNameDeParas, 'r', encoding='UTF-8', newline='\n')
    for dePara in deParas.readlines():
        dePara = dePara.split('(-*-)')
        input = re.sub(dePara[0], dePara[1], input, re.IGNORECASE, re.MULTILINE)
    return input

- Remove as stops Words (explicar brevemente oq é stop word)

In [ ]:
def removeStopWords(input):
    stopWrods = open(filenameStopWords, 'r', encoding='UTF-8', newline='\n')
    for stopWord in stopWrods.readlines():
        stopWord = unidecode.unidecode(stopWord)
        stopWord = re.sub('\n', '', stopWord)
        input = re.sub(stopWord, ' ', input, re.IGNORECASE, re.MULTILINE)
    return input

- Remove todas acentuaçoes(explicar para que)

In [18]:
def removeAcentuacao(input):
    return unidecode.unidecode(input)

In [15]:
dataSet = pd.read_excel(filenameDataSet, sheet_name='SEM RT')


In [16]:
listAux = []

for line in dataSet.values:
    line[6] = dePara(removeStopWords(removeLinesBreaks(unidecode.unidecode(toLower(line[6])))))
    listAux.append(line)
    
    
dataSet = pd.DataFrame(listAux)
